<!-- Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Projetos de Análise de Dados com Linguagem Python</font>
# <font color='blue'>Projeto 6 - Regras de Associação e Market Basket Analysis</font>

## Pacotes Python Usados no Projeto

In [1]:
!pip install -q -U watermark

https://pypi.org/project/efficient-apriori/

In [2]:
!pip install -q efficient_apriori

In [3]:
# Imports
import numpy as np
import pandas as pd
import efficient_apriori
import matplotlib.pyplot as plt
from datetime import datetime
from itertools import combinations
from efficient_apriori import apriori
import warnings
warnings.filterwarnings('ignore')

In [4]:
%reload_ext watermark
%watermark -a "Data Science Academy" 

Author: Data Science Academy



## Carregando os Dados

In [5]:
# Carrega os dados
dados_departamentos = pd.read_csv('departamentos.csv')
dados_corredores = pd.read_csv('corredores.csv')
dados_produtos = pd.read_csv('produtos.csv')
dados_pedidos = pd.read_csv('pedidos.csv')
dados_transacoes = pd.read_csv('transacoes.csv')

## Análise Exploratória

### Detalhes do Tamanho de Cada Dataset via Programação

In [6]:
# Total de registros por dataset
conta_arrays = np.array([['dados_departamentos', len(dados_departamentos)],
                        ['dados_corredores', len(dados_corredores)],
                        ['dados_produtos', len(dados_produtos)],
                        ['dados_pedidos', len(dados_pedidos)],
                        ['dados_transacoes', len(dados_transacoes)]])

In [7]:
# Converte o array em dataframe
contagem = pd.DataFrame(conta_arrays, columns = ['Arquivo', 'Total de Registros'])

In [8]:
print(contagem)

               Arquivo Total de Registros
0  dados_departamentos                 21
1     dados_corredores                134
2       dados_produtos              49688
3        dados_pedidos            3421083
4     dados_transacoes           32434489


In [9]:
dados_departamentos.shape

(21, 2)

In [10]:
dados_departamentos.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [11]:
dados_corredores.shape

(134, 2)

In [12]:
dados_corredores.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [13]:
dados_produtos.shape

(49688, 4)

In [14]:
dados_produtos.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [15]:
dados_pedidos.shape

(3421083, 7)

In [16]:
dados_pedidos.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [17]:
dados_transacoes.shape

(32434489, 4)

In [18]:
dados_transacoes.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


### Verificando Valores Ausentes

In [19]:
dados_departamentos.isna().sum()

department_id    0
department       0
dtype: int64

In [20]:
dados_corredores.isna().sum()

aisle_id    0
aisle       0
dtype: int64

In [21]:
dados_produtos.isna().sum()

product_id       0
product_name     0
aisle_id         0
department_id    0
dtype: int64

In [22]:
dados_pedidos.isna().sum()

order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

In [23]:
dados_transacoes.isna().sum()

order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64

## Reorganização dos Dados

### Merge das Tabelas

In [24]:
%%time
dados_transacoes = dados_transacoes.merge(dados_pedidos, on = 'order_id', how = 'left')
dados_transacoes = dados_transacoes.merge(dados_produtos, on = 'product_id', how = 'left')
dados_transacoes = dados_transacoes.merge(dados_corredores, on = 'aisle_id', how = 'left')
dados_transacoes = dados_transacoes.merge(dados_departamentos, on = 'department_id', how = 'left')

CPU times: user 5.34 s, sys: 1.33 s, total: 6.67 s
Wall time: 6.84 s


In [25]:
dados_transacoes.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,department
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,eggs,dairy eggs
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,fresh vegetables,produce
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,spices seasonings,pantry
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,oils vinegars,pantry
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,baking ingredients,pantry


In [26]:
dados_transacoes.isna().sum()

order_id                        0
product_id                      0
add_to_cart_order               0
reordered                       0
user_id                         0
eval_set                        0
order_number                    0
order_dow                       0
order_hour_of_day               0
days_since_prior_order    2078068
product_name                    0
aisle_id                        0
department_id                   0
aisle                           0
department                      0
dtype: int64

In [27]:
dados_transacoes.isnull().sum() / len(dados_transacoes) * 100

order_id                  0.00000
product_id                0.00000
add_to_cart_order         0.00000
reordered                 0.00000
user_id                   0.00000
eval_set                  0.00000
order_number              0.00000
order_dow                 0.00000
order_hour_of_day         0.00000
days_since_prior_order    6.40697
product_name              0.00000
aisle_id                  0.00000
department_id             0.00000
aisle                     0.00000
department                0.00000
dtype: float64

Vamos manter a variável days_since_prior_order por duas razões:

1- O percentual de valores ausentes é apenas de 6.4%.

2- Não usaremos a variável com o algoritmo MBA.

> Vamos agrupar os dados e preparar o dataset com todas as transações. Usaremos os dados para aplicar o algoritmo Apriori mais tarde.

In [28]:
%%time
df_dsa = pd.DataFrame(dados_transacoes.groupby('order_id')['product_id'])

CPU times: user 35.6 s, sys: 824 ms, total: 36.4 s
Wall time: 36.6 s


In [29]:
df_dsa.shape

(3214874, 2)

In [30]:
df_dsa.head()

,0,1
0,2,0 33120 1 28985 2 9327 3 45918 4 ...
1,3,9 33754 10 24838 11 17704 12 2190...
2,4,17 46842 18 26434 19 39758 20 2776...
3,5,30 13176 31 15005 32 47329 33 2796...
4,6,56 40462 57 15873 58 41897 Name: prod...


In [31]:
# Lista
prod = []

In [32]:
%%time
for i in range(len(df_dsa[0])):
    prod.append(list(df_dsa.iloc[i][1]))

CPU times: user 44.2 s, sys: 243 ms, total: 44.4 s
Wall time: 44.6 s


In [33]:
# Prepara o dataframe
transacoes = pd.DataFrame({'Id_Pedido' : df_dsa[0], 'Produtos' : prod})

In [34]:
transacoes.head(10)

,Id_Pedido,Produtos
0,2,"[33120, 28985, 9327, 45918, 30035, 17794, 4014..."
1,3,"[33754, 24838, 17704, 21903, 17668, 46667, 174..."
2,4,"[46842, 26434, 39758, 27761, 10054, 21351, 225..."
3,5,"[13176, 15005, 47329, 27966, 23909, 48370, 132..."
4,6,"[40462, 15873, 41897]"
5,7,"[34050, 46802]"
6,8,[23423]
7,9,"[21405, 47890, 11182, 2014, 29193, 34203, 1499..."
8,10,"[24852, 4796, 31717, 47766, 4605, 1529, 21137,..."
9,11,"[30162, 27085, 5994, 1313, 31506]"


## Agora é Com Você!

Responda as perguntas abaixo através de resumos, tabelas e gráficos.

### 1- Qual o Número de Pedidos Mais Frequente Entre os Usuários?

### 2- Qual Dia da Semana Tem o Maior Número de Pedidos?

### 3- Qual Hora do Dia Tem o Maior Número de Pedidos?

### 4- Qual Departamento Tem o Maior Número de Pedidos?

### 5- Quais os 20 Principais Corredores Por Frequência de Pedidos?

### 6- Quais os 20 Principais Produtos Por Frequência de Pedidos?

### 7- Faça a Análise de Novos Pedidos Semelhantes a Pedidos Anteriores (Reorder)

### 8- Faça a Análise de Reorder Por Departamento ao Longo do Tempo

### 9- Faça a Análise de Reorder e Pedidos

### 10- Faça a Análise de Reorder Por Corredor

### 10.1- Faça a Análise de Reorder Por Corredor (Total)

As soluções serão apresentadas no próximo capítulo.

In [35]:
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [36]:
#%watermark -v -m

In [37]:
#%watermark --iversions

# Fim